<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/Notebooks/spark_level_1_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BÀI TẬP SPARK – LEVEL 1:



## 🔹 Bài 1: Tạo DataFrame

### 🎯 Mục tiêu:
>*	Hiểu schema:
>*	Nhận biết kiểu dữ liệu

In [3]:
# check môi trường:
try:
    spark
    print("✅ Spark đã tồn tại")
except NameError:
    print("❌ Spark CHƯA được tạo")

❌ Spark CHƯA được tạo


In [4]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [5]:
!java --version

openjdk 17.0.17 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [6]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2025-12-29 06:57:05--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  16.6MB/s    in 70s     

2025-12-29 06:58:16 (7.46 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [7]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


In [9]:
data = [
    (1, "A", 100),
    (2, "A", 200),
    (3, "B", 300),
    (4, "B", 400),
    (5, "C", 500),
]

df = spark.createDataFrame(data, ["id", "group", "amount"])
df.show()
df.printSchema()

+---+-----+------+
| id|group|amount|
+---+-----+------+
|  1|    A|   100|
|  2|    A|   200|
|  3|    B|   300|
|  4|    B|   400|
|  5|    C|   500|
+---+-----+------+

root
 |-- id: long (nullable = true)
 |-- group: string (nullable = true)
 |-- amount: long (nullable = true)



### 🔹 Bài 2: Transformation vs Action

### 🎯 Mục tiêu:
>*	Hiểu lazy execution
>*	Biết Spark chỉ chạy khi có action

In [10]:
# chưa chạy gỉ cả
df2 = df.filter(df.amount > 200)

In [11]:
df2.explain()

== Physical Plan ==
*(1) Filter (isnotnull(amount#15L) AND (amount#15L > 200))
+- *(1) Scan ExistingRDD[id#13L,group#14,amount#15L]




In [12]:
df2.show()

+---+-----+------+
| id|group|amount|
+---+-----+------+
|  3|    B|   300|
|  4|    B|   400|
|  5|    C|   500|
+---+-----+------+



### 🔹 Bài 3: Shuffle đầu tiên (groupBy)

```python
from pyspark.sql.functions import sum

df.groupBy("group").agg(sum("amount")).show()
```

```python
df.groupBy("group").agg(sum("amount")).explain()
```

👉 Tìm dòng:

```code
Exchange hashpartitioning
```

🎯 Đây chính là shuffle

In [13]:
from pyspark.sql.functions import sum

df.groupBy("group").agg(sum("amount")).show()

+-----+-----------+
|group|sum(amount)|
+-----+-----------+
|    A|        300|
|    B|        700|
|    C|        500|
+-----+-----------+



In [14]:
df.groupBy("group").agg(sum("amount")).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[group#14], functions=[sum(amount#15L)])
   +- Exchange hashpartitioning(group#14, 4), ENSURE_REQUIREMENTS, [plan_id=107]
      +- HashAggregate(keys=[group#14], functions=[partial_sum(amount#15L)])
         +- Project [group#14, amount#15L]
            +- Scan ExistingRDD[id#13L,group#14,amount#15L]




### 🔹 Bài 4: Partition

👉 Ghi chú:
>*	repartition = shuffle
>*	coalesce = không shuffle (khi giảm)

In [15]:
df.rdd.getNumPartitions()

2

In [16]:
df2 = df.repartition(4)
df2.rdd.getNumPartitions()

4

### 🔹 Bài 5: Ghi file (chuẩn Bronze)

In [17]:
df.write.mode("overwrite").parquet("/content/bronze/orders")

In [18]:
spark.read.parquet("/content/bronze/orders").show()

+---+-----+------+
| id|group|amount|
+---+-----+------+
|  3|    B|   300|
|  4|    B|   400|
|  5|    C|   500|
|  1|    A|   100|
|  2|    A|   200|
+---+-----+------+

